[![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239436-lesson-5-chatbot-w-summarizing-messages-and-memory)

# 具有消息摘要功能的聊天机器人

## 回顾

我们已经介绍了怎样自定义图状态模式和汇总器 reducer。 
 
我们还展示了多种裁剪或过滤图状态中消息的方法。

## 目标

现在，让我们更进一步

不仅仅是裁剪和过滤消息，我们将展示怎样使用LLMs来生成对话的实时摘要。
 
这允许我们保留整个对话的一个压缩表示，而不是仅仅是通过裁剪或过滤删除它。

我们将把这个摘要集成到一个简单的聊天机器人中。

我们还将为这个聊天机器人装备记忆功能，支持长时间对话而不会产生高额token消耗/延迟。 

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_openai

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [1]:
# 我没有openai的api key，所以使用qwen的openai兼容接口，需要设置qwen的api key和base url，我选择从配置文件.yml中读取
import yaml

with open("../.yml", "r") as f:
    config = yaml.safe_load(f)

qwen_config = config["llm"]["qwen"]
    

我们将使用 [LangSmith](https://docs.smith.langchain.com/) 进行 [追踪](https://docs.smith.langchain.com/concepts/tracing).

In [3]:
import os

# _set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = config["langsmith"]["key"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-study"

In [26]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="qwen-max",api_key=qwen_config["api_key"],base_url=qwen_config["base_url"],temperature=0.1)

像之前一样，我们将使用 `MessagesState`。

除了内置的 `messages` 键，我们现在将引入一个自定义的键（`summary`）。

In [5]:
from langgraph.graph import MessagesState
class State(MessagesState):
    summary: str

我们将定义一个调用我们LLM的节点，如果存在摘要，将其纳入提示词中。

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

# Define the logic to call the model
def call_model(state: State):
    
    # Get summary if it exists
    summary = state.get("summary", "")

    # If there is summary, then we add it
    if summary:
        
        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]
    
    else:
        messages = state["messages"]
    
    response = model.invoke(messages)
    return {"messages": response}

我们将定义一个节点来生成摘要。

注意，这里我们将在生成摘要后使用 `RemoveMessage` 来过滤我们的状态。

In [43]:
def summarize_conversation(state: State):
    
    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt 
    if summary:
        
        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above, only output the summary, no other text:"
        )
        
    else:
        summary_message = "Create a summary of the conversation above, only output the summary, no other text:"

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

我们将添加一个条件边来根据对话长度决定是否生成摘要。

In [8]:
from langgraph.graph import END
# Determine whether to end or summarize the conversation
def should_continue(state: State):
    
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    
    # Otherwise we can just end
    return END

## 添加记忆

回忆知识点：对一次单个图执行来说[状态是瞬时的](https://github.com/langchain-ai/langgraph/discussions/352#discussioncomment-9291220) 。

这限制了我们拥有可中断的多轮对话的能力。

正如 Module 1 结尾所介绍的，我们可以使用[持久化](https://langchain-ai.github.io/langgraph/how-tos/persistence/)来解决这个问题。
 
LangGraph 可以使用一个检查器在每一步后自动保存图状态。

这个内置的持久化层给我们提供了记忆，允许 LangGraph 从最后一次状态更新处恢复。

正如我们之前所展示的，最容易使用的一个是 `MemorySaver`，一个图状态的内存键值对存储。

我们所需要做的就是用一个检查器来编译图，我们的图就有了记忆！

In [44]:
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START

# Define a new graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)
# display(Image(graph.get_graph().draw_mermaid_png()))
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	conversation(conversation)
	summarize_conversation(summarize_conversation)
	__end__([<p>__end__</p>]):::last
	__start__ --> conversation;
	summarize_conversation --> __end__;
	conversation -.-> summarize_conversation;
	conversation -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



## Threads

检查器在每一步后保存状态作为检查点。

这些保存的检查点可以被分组到一个对话的 `thread` 中。

将Slack（一款团队协作沟通工具类似飞书吧）当做类比：不同的频道承载了不同的对话。

![slack.png](./slack.png)

Threads 就像 Slack 的频道, 捕捉分组的状态集合（例如，对话）。

下边，我们使用 `configurable` 来设置 thread ID.

![state.jpg](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbadf3b379c2ee621adfd1_chatbot-summarization1.png)

In [46]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}

memory.storage.clear()

# Start conversation
input_message = HumanMessage(content="hi! I'm Lance")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="what's my name?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="i like the 49ers!")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Hello Lance! It's great to meet you. How can I assist you today? Whether you have questions, need information, or just want to chat about something, feel free to let me know!
================================== Ai Message ==================================

Your name is Lance! Is there anything else you'd like to know or discuss?
================================== Ai Message ==================================

That's great, Lance! The San Francisco 49ers have a rich history and a passionate fan base. Do you have a favorite player, or are you looking forward to any upcoming games? Or maybe you'd like to talk about some memorable moments in 49ers history?


现在，我们还没有状态的摘要，因为我们的消息仍然<=6条。

这是在 `should_continue` 中设置的。

```
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
```

我们可以恢复对话因为我们拥有 thread。

In [47]:
graph.get_state(config).values.get("summary","")

''

这个带有 thread ID 的 `config` 允许我们从之前记录的状态继续进行！

In [48]:
input_message = HumanMessage(content="i like Nick Bosa, isn't he the highest paid defensive player?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Yes, you're right! Nick Bosa is one of the top defensive players in the NFL, and he recently signed a massive contract extension with the San Francisco 49ers. In March 2023, he signed a five-year, $130 million deal, making him the highest-paid defensive player in the league at that time. His performance on the field has been outstanding, and he's a key part of the 49ers' defense.

Do you have any favorite moments or plays from Nick Bosa, or are you excited about how he'll perform this season?


In [49]:
graph.get_state(config).values.get("summary","")

'Lance introduced himself and mentioned he likes the San Francisco 49ers. He specifically noted his appreciation for Nick Bosa, who recently signed a five-year, $130 million contract, making him the highest-paid defensive player in the NFL.'

## LangSmith

让我们检查一下追踪信息。